In [813]:
import csv
import pandas as pd
import numpy as np
import operator

In [814]:
# Read airquality data
df = pd.read_csv('../data/airquality_aggregated.csv', parse_dates=['date'])

In [815]:
# Filter to a specific year
year = 2019
df = df[df.date.dt.year == year]

In [816]:
df.head()

,date,county,latitude,longitude,measure
163538,2019-01-01,Alameda,37.767498,-122.117606,4.642476
163539,2019-01-01,Butte,39.761680,-121.840470,13.500000
163540,2019-01-01,Calaveras,38.201850,-120.680277,17.903333
163541,2019-01-01,Contra Costa,37.948206,-122.191482,3.716140
163542,2019-01-01,Fresno,36.805813,-119.842293,34.574722


In [817]:
# Find all counties
df_counties = pd.read_csv('../data/county_lat_long.csv')
counties = df_counties.County.unique()
df_counties.set_index(['County'], inplace=True)
print(counties)
print(len(counties))

['Alameda' 'Alpine' 'Amador' 'Butte' 'Calaveras' 'Colusa' 'Contra Costa'
 'Del Norte' 'El Dorado' 'Fresno' 'Glenn' 'Humboldt' 'Imperial' 'Inyo'
 'Kern' 'Kings' 'Lake' 'Lassen' 'Los Angeles' 'Madera' 'Marin' 'Mariposa'
 'Mendocino' 'Merced' 'Modoc' 'Mono' 'Monterey' 'Napa' 'Nevada' 'Orange'
 'Placer' 'Plumas' 'Riverside' 'Sacramento' 'San Benito' 'San Bernardino'
 'San Diego' 'San Francisco' 'San Joaquin' 'San Luis Obispo' 'San Mateo'
 'Santa Barbara' 'Santa Clara' 'Santa Cruz' 'Shasta' 'Sierra' 'Siskiyou'
 'Solano' 'Sonoma' 'Stanislaus' 'Sutter' 'Tehama' 'Trinity' 'Tulare'
 'Tuolumne' 'Ventura' 'Yolo' 'Yuba']
58


In [818]:
# Calculate which counties don't have 90 days of data
df_sparse_counties = df.groupby('county')['measure'].count().rename('count').reset_index().query('count < 90')
print(df_sparse_counties)

    county  count
11    Lake     60
36  Shasta     53
45    Yolo     53


In [819]:
# Replace the aqi values of sparse counties with nan
for index, row in df_sparse_counties.iterrows():
    df.loc[df['county'] == row['county'], 'measure'] = np.nan
print(df)

             date        county   latitude   longitude    measure
163538 2019-01-01       Alameda  37.767498 -122.117606   4.642476
163539 2019-01-01         Butte  39.761680 -121.840470  13.500000
163540 2019-01-01     Calaveras  38.201850 -120.680277  17.903333
163541 2019-01-01  Contra Costa  37.948206 -122.191482   3.716140
163542 2019-01-01        Fresno  36.805813 -119.842293  34.574722
...           ...           ...        ...         ...        ...
178650 2019-12-31        Sonoma  38.403765 -122.818294   8.105000
178651 2019-12-31    Stanislaus  37.565241 -120.915110  20.161667
178652 2019-12-31        Sutter  39.138773 -121.618549  18.100000
178653 2019-12-31        Tehama  40.170930 -122.255560   9.411667
178654 2019-12-31       Ventura  34.311255 -118.903723   1.509722

[15117 rows x 5 columns]


In [820]:
# Generate dataframe with rows for every day/county
all_days = pd.date_range(start=df.date.min(), end=df.date.max()).strftime('%Y-%m-%d')
mux = pd.MultiIndex.from_product((all_days, counties, [np.nan]), names=['date', 'county', 'measure'])
df_final = mux.to_frame(index=False)
df_final.set_index(['date', 'county'], inplace=True)
print(df_final)

                      measure
date       county            
2019-01-01 Alameda        NaN
           Alpine         NaN
           Amador         NaN
           Butte          NaN
           Calaveras      NaN
...                       ...
2019-12-31 Tulare         NaN
           Tuolumne       NaN
           Ventura        NaN
           Yolo           NaN
           Yuba           NaN

[21170 rows x 1 columns]


In [821]:
# Add known data to df_final
for index, row in df.iterrows():
    df_final.loc[row['date'].strftime('%Y-%m-%d'),row['county']] = row['measure']
df_final.head()

measure
date       county              
2019-01-01 Alameda     4.642476
           Alpine           NaN
           Amador           NaN
           Butte      13.500000
           Calaveras  17.903333

In [822]:
# Calculate counties with missing values
df_missing_counties = df_final.groupby('county').count().reset_index().query('measure == 0').county
print(df_missing_counties)

1        Alpine
2        Amador
7     Del Norte
8     El Dorado
10        Glenn
16         Lake
17       Lassen
21     Mariposa
24        Modoc
44       Shasta
45       Sierra
52      Trinity
54     Tuolumne
56         Yolo
57         Yuba
Name: county, dtype: object


In [823]:
# For all rows with some, but not all data being NaN, extrapolate values based on nearest dates

In [824]:
## Set data for the min date
missing_min_date = df_final.loc[df.date.min().strftime('%Y-%m-%d')].query('measure.isnull()').index.tolist()
for county in missing_min_date:
    # Only look at counties with some data
    if not county in list(df_missing_counties):
        # Find the earliest date with aqi values and set that value for the min date
        min_date_aqi = df_final.query('county == @county and not measure.isnull()').iloc[0].measure
        df_final.loc[df.date.min().strftime('%Y-%m-%d'), county] = min_date_aqi

## Set data for the max date
missing_max_date = df_final.loc[df.date.max().strftime('%Y-%m-%d')].query('measure.isnull()').index.tolist()
for county in missing_max_date:
    # Only look at counties with some data
    if not county in list(df_missing_counties):
        # Find the earliest date with aqi values and set that value for the max date
        max_date_aqi = df_final.query('county == @county and not measure.isnull()').iloc[-1].measure
        df_final.loc[df.date.max().strftime('%Y-%m-%d'), county] = max_date_aqi

In [825]:
## Extrapolate date values in between
for county in counties:
    # Only look at counties with some data
    if not county in list(df_missing_counties):
        for index, row in df_final.query('county == @county').iterrows():
            if np.isnan(row.measure):
                before = df_final.query('county == @county and date < @index[0]  and not measure.isnull()').iloc[-1]
                after = df_final.query('county == @county and date > @index[0]  and not measure.isnull()').iloc[0]

                before_date = datetime.strptime(before.name[0], '%Y-%m-%d')
                missing_date = datetime.strptime(index[0], '%Y-%m-%d')
                after_date = datetime.strptime(after.name[0], '%Y-%m-%d')

                before_diff = (missing_date-before_date).days
                after_diff = (after_date-missing_date).days
                average = before.measure * (1-before_diff/(before_diff+after_diff)) + after.measure * (1-after_diff/(before_diff+after_diff))

                df_final.loc[index[0], county] = average
            
    

In [826]:
# For all rows will all NaN, find two nearest counties and average those values
def find_nearest(c):
    lat = df_counties.loc[c].Latitude
    long = df_counties.loc[c].Longitude
    distances = {}
    for n in df_counties.query('County != @c').index.tolist():
        if not n in list(df_missing_counties):
            distances[n] = (df_counties.loc[n].Latitude - lat)**2+(df_counties.loc[n].Longitude - long)**2
    dist_sorted = sorted(distances.items(), key=operator.itemgetter(1))[:3]
    return list(map(lambda x: x[0], dist_sorted))
    
for county in list(df_missing_counties):
    nearby = find_nearest(county)
    for date in all_days:
        ave = 0
        for near in nearby:
            ave = ave + df_final.loc[date, near]
        df_final.loc[date, county] = ave / 3
print(df_final)

                        measure
date       county              
2019-01-01 Alameda     4.642476
           Alpine     11.660417
           Amador      8.819064
           Butte      13.500000
           Calaveras  17.903333
...                         ...
2019-12-31 Tulare     10.700000
           Tuolumne   20.178056
           Ventura     1.509722
           Yolo       13.786957
           Yuba       13.527099

[21170 rows x 1 columns]


In [827]:
# Write resulting data to csv
df_final.to_csv('../data/' + str(year) + '.csv')